### Spring JDBC
Spring JDBC is an abstraction over Java JDBC and does the following things for us:
- Open the connection
- Prepare and run the statement
- Set up the loop to iterate through the results (if any)
- Process and handle any exception
- Close the connection, the statement, and the resultset

Spring JDBC provides a few different DataSource implementations like `SingleConnectionDataSource`, `DriverManagerDataSource`, etc. In order to connect with a database, we first need a DataSource

In [ ]:
@Bean("mysql-datasource")
public DataSource mysqlDataSource() {
    DriverManagerDataSource dataSource = new DriverManagerDataSource();
    dataSource.setUrl("jdbc:mysql://localhost:3306/employees?autoReconnect=true&useSSL=false");
    dataSource.setUsername("root");
    dataSource.setPassword("root");

    return dataSource;
}

`DriverManagerDataSource` is provided by Spring JDBC. This implementation does not provide any connection pooling. Spring provides `JdbcTemplate` object which is the main object responsible for executing sql statements.

In [ ]:
ApplicationContext context = new AnnotationConfigApplicationContext(ApplicationConfiguration.class);

DataSource ds = context.getBean("mysql-datasource", DataSource.class);
JdbcTemplate template = new JdbcTemplate(ds);

### Querying Database
**Non-parameterised query with 1 result:** `IncorrectResultSizeDataAccessException` is thrown if more than one row is fetched

In [ ]:
// Simple objects
String countSql = "SELECT COUNT(*) FROM employees";
int employeeCount = template.queryForObject(countSql, Integer.class);

String typeSql = "SELECT `Type 1` FROM pokemon WHERE name=\"Bulbasaur\"";
String name = template.queryForObject(typeSql, String.class);

For complex objects, we make use of `RowMapper` interface

In [ ]:
String pokemonSql = "SELECT * FROM pokemon WHERE name=\"Bulbasaur\"";
Pokemon bulbasaur = template.queryForObject(pokemonSql, new RowMapper<Pokemon>() {

    public Pokemon mapRow(ResultSet rs, int rowNum) throws SQLException {
        Pokemon pokemon = new Pokemon();
        pokemon.name = rs.getString("name");
        pokemon.type1 = rs.getString("Type 1");
        pokemon.type2 = rs.getString("Type 2");
        pokemon.hp = rs.getInt("hp");
        pokemon.attack = rs.getInt("attack");
        pokemon.defense = rs.getInt("defense");

        return pokemon;
    }

});

**Parameterised query with 1 result:** use an overloaded version of queryForObject

In [ ]:
// Single parameter (last argument)
int countOfActorsNamedJoe = this.jdbcTemplate.queryForObject(
        "SELECT COUNT(*) FROM actors WHERE first_name = ?", Integer.class, "Joe");

// Another way to pass single parameter
// This method is more suited for passing multiple parameters
String pokemonSql = "SELECT * FROM pokemon WHERE name= ?";
Pokemon raticate = template.queryForObject(pokemonSql, new Object[] { "Raticate" }, new RowMapper<Pokemon>() {

    public Pokemon mapRow(ResultSet rs, int rowNum) throws SQLException {
        Pokemon pokemon = new Pokemon();
        pokemon.name = rs.getString("name");
        pokemon.type1 = rs.getString("Type 1");
        pokemon.type2 = rs.getString("Type 2");
        pokemon.hp = rs.getInt("hp");
        pokemon.attack = rs.getInt("attack");
        pokemon.defense = rs.getInt("defense");

        return pokemon;
    }

});

**Multiple Results:** 

In [ ]:
String pokemonSql = "SELECT * FROM pokemon WHERE hp > ?";
List<Pokemon> pokemons = template.query(pokemonSql, new Object[] { 100 }, new RowMapper<Pokemon>() {

    public Pokemon mapRow(ResultSet rs, int rowNum) throws SQLException {
        Pokemon pokemon = new Pokemon();
        pokemon.name = rs.getString("name");
        pokemon.type1 = rs.getString("Type 1");
        pokemon.type2 = rs.getString("Type 2");
        pokemon.hp = rs.getInt("hp");
        pokemon.attack = rs.getInt("attack");
        pokemon.defense = rs.getInt("defense");

        return pokemon;
    }

});

/ 

If the result contains high number of rows it can lead to memory error. The below technique can be used to iterate over resultset.

In [ ]:
template.query(pokemonSql, new Object[] { 100 }, new RowCallbackHandler() {

    public void processRow(ResultSet rs) throws SQLException {
        System.out.println("Pokemon name is " + rs.getString("name"));
    }
});

### Updating
Use the `update` method to execute insert, delete or update queries. The update method returns the number of rows affected.

In [ ]:
String delSql = "DELETE FROM actors WHERE id = ?"
int deleteCount = template.update(delSql, 45);

String insSql = "INSERT INTO actors (first_name, last_name) VALUES (?, ?)"
int insCount = template.update(insSql, "Steve", "Jobs");

### Batch Update
One way is to use `BatchPreparedStatementSetter`. The `batchUpdate` method returns an integer array.

In [ ]:
final List<Pokemon> pokemonList = pokemonRepo.findAll();
String batchSql = "INSERT INTO pokemon (name, hp) VALUES (?, ?)";
template.batchUpdate(batchSql, new BatchPreparedStatementSetter() {

    public void setValues(PreparedStatement ps, int i) throws SQLException {
        ps.setString(1, pokemonList.get(i).name);
        ps.setInt(2, pokemonList.get(i).hp);
    }

    public int getBatchSize() {
        return pokemonList.size();
    }
});

Another way:

In [ ]:
final List<Pokemon> pokemonList = pokemonRepo.findAll();

List<Object[]> params = new ArrayList<Object[]>();
for (Pokemon p : pokemonList) {
    params.add(new Object[] { p.name, p.hp });
}

String batchSql = "INSERT INTO pokemon (name, hp) VALUES (?, ?)";
template.batchUpdate(batchSql, params);

### Exception Handling
The query and update methods do not throw any checked exception. In order to catch exception:

In [ ]:
try {
    // Execute, query, update statements
} catch (DataAccessException e) {
    // Handle the exception
}

DataAccessException is thrown rather than SQLException